## Statemachine test

In [1]:
%load_ext autoreload
%autoreload 2
import pandas as pd
import numpy as np

In [2]:
import dmyplant2
dmyplant2.cred()
mp = dmyplant2.MyPlant(600)
e=dmyplant2.Engine.from_sn(mp,1184199)
# print(f"{e}")
e.dash
# dtrips = e.batch_hist_alarms(p_severities=[600,650,700,800], p_offset=0, p_limit=200000)
# dtrips = dtrips.iloc[::-1] # Alarme umdrehen 
# dtrips.to_pickle('1184199_messages.pkl')
# manual = e.batch_hist_alarms(p_severities=[600], p_offset=0, p_limit=200000)
# manual = manual.iloc[::-1] # Alarme umdrehen 
# manual.to_pickle('1184199_manual.pkl')
# manual = pd.read_pickle('1184199_manual.pkl').reset_index()
# manual.head()

{'Name': 'ROCHE PENZBERG KWKK',
 'Engine ID': 'M01',
 'Design Number': 'U233',
 'Engine Type': '624',
 'Engine Version': 'H02',
 'P': 24,
 'P_nom': 4489.3,
 'BMEP': 24.0,
 'serialNumber': '1184199',
 'id': 108532,
 'Count_OpHour': 39422.0,
 'val start': Timestamp('2016-12-22 00:00:00'),
 'oph@start': 0,
 'oph parts': 39422,
 'LOC': 0.014}

In [3]:
#from dfsm import msgFSM
messages = pd.read_pickle('1184199_messages.pkl').reset_index()
messages.shape

(127539, 8)

In [26]:
# with open('msg.txt', 'w') as f:
#     for index, msg in messages.iterrows():
#         f.write(f"{index:>06} {msg['severity']} {pd.to_datetime(int(msg['timestamp'])*1e6).strftime('%d.%m.%Y %H:%M:%S')} {msg['name']} {msg['message']}\n")

In [ ]:
# with open('manual.txt', 'w') as f:
#     for index, msg in manual.iterrows():
#         f.write(f"{index:>06} {msg['severity']} {pd.to_datetime(int(msg['timestamp'])*1e6).strftime('%d.%m.%Y %H:%M:%S')} {msg['name']} {msg['message']}\n")

In [4]:
for index, msg in messages[:100000].iterrows():
    if str(msg['message']).startswith('Ready'):
    #if str(msg['message']).startswith('Cold start CPU'):
    #if str(msg['message']).startswith('Service selector'):
        pass
        #print(f"{index:>06} {msg['severity']} {pd.to_datetime(int(msg['timestamp'])*1e6).strftime('%d.%m.%Y %H:%M:%S')} {msg['name']} {msg['message']}")

In [10]:
from IPython.display import HTML, display
m = messages[messages.name != '9007'] # filter out hourly messages
m = m[:]
display(HTML(m[:5].to_html(escape=False)))

,index,assetId,isPrimary,message,name,severity,timestamp,associatedValues
0,127538,108532,NaN,Ignition point optimization,2141,600,1466768043798,NaN
1,127537,108532,NaN,Cold start CPU,1254,600,1466768043798,NaN
2,127536,108532,NaN,Misfire controller shutdown,2876,600,1466768043799,NaN
3,127535,108532,NaN,Speed setpoint input - internal (DIA.NE),2136,600,1466768043899,NaN
4,127534,108532,NaN,Power setpoint input - internal (DIA.NE),2133,600,1466768043899,NaN


In [12]:
from dfsm import Start_FSM
from pprint import pprint as pp
result = pd.DataFrame(Start_FSM(m))


,warnings,alarms,operations,mode,msg
0,[],[],"[[127537, 108532, nan, Cold start CPU, 1254, 6...",FSM-start,index 127537 assetId ...
1,[],[],"[[126441, 108532, nan, Service selector switch...",mode-manual,index 12...
2,"[[126421, 108532, False, Measuring signal fail...","[[126423, 108532, nan, Gas alarm, 1053, 800, 1...","[[126431, 108532, nan, Demand oil pump on, 125...",mode-off,index 12639...
3,"[[126421, 108532, False, Measuring signal fail...","[[126423, 108532, nan, Gas alarm, 1053, 800, 1...","[[126431, 108532, nan, Demand oil pump on, 125...",mode-off,index 12639...
4,[],[],"[[126389, 108532, nan, Service selector switch...",mode-automatic,index ...
...,...,...,...,...,...
2801,"[[146, 108532, False, Lifetime reached - NOx s...","[[36, 108532, False, Jacket water pressure low...","[[279, 108532, False, Demand oil pump on, 1259...",mode-off,index 3...
2802,"[[146, 108532, False, Lifetime reached - NOx s...","[[36, 108532, False, Jacket water pressure low...","[[279, 108532, False, Demand oil pump on, 1259...",mode-off,index 3...
2803,[],[],"[[13, 108532, False, Service selector switch M...",mode-manual,index ...
2804,[],[],"[[8, 108532, False, Service selector switch Of...",mode-off,index ...


In [14]:

result.iloc[0]['operations']

[index                       127537
 assetId                     108532
 isPrimary                      NaN
 message             Cold start CPU
 name                          1254
 severity                       600
 timestamp            1466768043798
 associatedValues               NaN
 Name: 1, dtype: object]